<a href="https://colab.research.google.com/github/Tymiec/Semantyczny_Internet_2022-2023/blob/master/semantic_mashups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sparqlwrapper rdflib owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 29.8 MB/s 
     |████████████████████████████████| 25.5 MB 3.2 MB/s 
     |████████████████████████████████| 41 kB 591 kB/s 
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260904 sha256=fefa5ddb334687bf34b4befbdd12b047f874dbe6288c8fc037f361e50a7ba822
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


In [ ]:
import SPARQLWrapper
import rdflib
import requests
import json
import pandas as pd
from google.colab import drive

PATH = '/content/drive/MyDrive/uj/data/f1'

In [ ]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
drivers_df = pd.read_csv(PATH + '/drivers.csv')

drivers_df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [ ]:
def get_wikipedia_pageids(link):
  page_title = link.split("/")[-1]
  api_endpoint = f"https://en.wikipedia.org/w/api.php?action=query&format=json&titles={page_title}"
  response = requests.get(api_endpoint)
  data = json.loads(response.text)
  ids = data["query"]["pages"].keys()
  return list(ids)

In [ ]:
hamilton_id = get_wikipedia_pageids('https://en.wikipedia.org/wiki/Lewis_Hamilton')[0]
print(hamilton_id)

675561


In [ ]:
wikidata = SPARQLWrapper.SPARQLWrapper("https://query.wikidata.org/sparql")
dbpedia = SPARQLWrapper.SPARQLWrapper('https://dbpedia.org/sparql')

dbpedia.setQuery(f"""
PREFIX dbo: http://dbpedia.org/ontology/
PREFIX dbr: http://dbpedia.org/resource/

SELECT ?driver ?driverName WHERE {{
  ?driver a dbo:FormulaOneRacer ;
          dbo:wikiPageID '675561' ;
          rdfs:label ?driverName .
  FILTER (lang(?driverName) = "en")
}}
""")

dbpedia.setReturnFormat(SPARQLWrapper.JSON)
wd_results = dbpedia.query().convert()

print(wd_results)

# dbpedia.setReturnFormat(SPARQLWrapper.RDFXML)
# wd_results = dbpedia.query()
# wd_xml_response = wd_results.response.read()

# graph = rdflib.Graph()
# graph.parse(data=wd_xml_response, format="xml")

QueryBadFormed: ignored

In [ ]:
# now, you can go through the results (it is a Python dict!) and print them
print("TOP 5 countries by population:")
for result in wd_results["results"]["bindings"]:
    print(f'{result["countryLabel"]["value"]} ({result["country"]["value"]}): ' +
          f'{result["population"]["value"]}')

In [ ]:
# Prepare the SPARQLWrapper for DBpedia
dbpedia = # _FILL_IN_YOUR_CODE_HERE

def getLeadersForCountry(wikidataURI):
    # Prepare the query
    dbpedia.setQuery(  # _FILL_IN_YOUR_CODE_HERE
                    )

    # query will be executed, the result will be parsed into one string and returned
    dbp_results = dbpedia.query().convert()
    leaders_string = ', '.join([result["leader"]["value"] for result in dbp_results["results"]["bindings"]])
    return leaders_string

In [ ]:
print("Leaders of TOP 5 countries by population:")
for result in wd_results["results"]["bindings"]:
    # execute the getLeadersForCountry() properly!
    print(f'{result["countryLabel"]["value"]}: {       # _FILL_IN_YOUR_CODE_HERE       }')

### Store graphs using `RDFLib`

*Use https://rdflib.readthedocs.io/en/stable/index.html as a reference for `RDFLib`*

But what if we want to keep a knowledge graph in Python?  
Let's extract the same knowledge from the Wikidata, but now as a graph (using the CONSTRUCT query)

In [ ]:
# specify the query
wikidata.setQuery("""
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ex: <https://example.org/>

CONSTRUCT {
    ?country  rdfs:label     ?countryLabel ;
              ex:population  ?population .
}
WHERE {
    ?country wdt:P31 wd:Q6256 ;
             wdt:P1082 ?population ;
             rdfs:label ?countryLabel .
    FILTER(LANG(?countryLabel) = "en")
}
ORDER BY DESC(?population)
LIMIT 5
""")

In [ ]:
# use the RDF/XML format (we will get the graph, so the JSON is not appropriate)
wikidata.setReturnFormat(SPARQLWrapper.RDFXML)

In [ ]:
# get the results, but do not convert them yet
wd_results = wikidata.query()
# instead, read the returned RDF/XML
wd_xml_response = wd_results.response.read()
# and show it on the screen
print(wd_xml_response)

Ok, we have the graph. But what to do now?  
We can use RDFLib to store this graph!

In [ ]:
import rdflib

In [ ]:
# Load the graph
graph = rdflib.Graph()
graph.parse(data=wd_xml_response, format="xml")

In [ ]:
# Now, we can print the whole graph in Turtle format
print(graph.serialize(format='turtle'))

You can add new triples to the graph using `add()` method:

In [ ]:
graph.add((
    rdflib.URIRef("http://www.wikidata.org/entity/Q36"),  # URIRef gives us possibility to provide any URI
    rdflib.RDFS.label,                                    # There are also some predefined URIS
    rdflib.Literal("Poland", lang="en")                   # Literals are also supported
))

graph.add((
    rdflib.URIRef("http://www.wikidata.org/entity/Q36"),
    rdflib.URIRef("https://example.org/population"),
    rdflib.Literal(38411148, datatype=rdflib.XSD.decimal)
))

In [ ]:
# Let's check if Poland is there
print(graph.serialize(format='turtle'))

**Task 2**: Fill the graph with information about leaders.
* Use the similar query as in Task 1 to extract the knowledge from DBpedia. Now, use the `SPARQL CONSTRUCT` scheme.
* Add the data in proper way. As a result, each country should have one `rdfs:label`, one `ex:population` and some `ex:leader` properties.
* To add a set of triples to the graph, you can simply use the `graph.parse()` method (as presented in: https://rdflib.readthedocs.io/en/stable/merging.html)

In [ ]:
dbpedia.setReturnFormat(SPARQLWrapper.RDFXML)

# the loop iterates over all subjects of triples with predicate ex:population,
# i.e., it iterates over all countries!
for wikidataURI in graph.subjects(predicate=rdflib.URIRef("https://example.org/population")):

    dbpedia.setQuery(   # _FILL_IN_YOUR_CODE_HERE     )
    
    # add new triples to the graph
    # _FILL_IN_YOUR_CODE_HERE
    

In [ ]:
# Let's check if everything is OK!
print(graph.serialize(format='turtle'))

**Task 3**: Extract something from your knowledge graph stored in `graph` using both ways provided by the RDFLib:
1. A set of Python methods (described in https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html)
2. A SPARQL interface (described in https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html)

You can also interact with your own graph instead of countries and leaders example if you wish :-)

In [ ]:
# _FILL_IN_YOUR_CODE_HERE

### Ontologies are also supported in Python (via `owlready2`)

*Use https://owlready2.readthedocs.io/en/latest/index.html as a reference for `owlready2`*

In [ ]:
import owlready2

In [ ]:
# download the FHKB ontology (used in reasoning lab: https://wiki.iis.uj.edu.pl/courses:semint:lab_reasoning)
!wget https://krzysztof.kutt.pl/didactics/semweb/fhkb.owl

In [ ]:
onto = owlready2.get_ontology("fhkb.owl")

In [ ]:
onto.load()

In [ ]:
# This is the base IRI for the ontology
print(onto.base_iri)

In [ ]:
# We can list all classes in the ontology (there are similar methods for properties and individuals)
print(list(onto.classes()))

**Task 4**: There is a Person class. Let's investigate it more.
* List all methods and fields available for `onto.Person` -- use built-in [`dir()`](https://docs.python.org/3/library/functions.html#dir) method
* The most interesting ones are: `name`, `iri`, `is_a`, `instances()` and `subclasses()` -- check them!

In [ ]:
# _FILL_IN_YOUR_CODE_HERE


**Task 5**: The ontology has a very useful search() function. See: https://owlready2.readthedocs.io/en/latest/onto.html#simple-queries  
 Use it to perform four queries:
1. search for all instances of `onto.Person`
2. search for all instances that are in relation `onto.hasFather` to any object
3. search for all instances that has `john` in the IRI
4. search for all instances that follow both: has `john` in the IRI and are in relation `onto.hasFather` to any object

In [ ]:
# _FILL_IN_YOUR_CODE_HERE


**Task 6:** You can simply iterate through the `search()` results, to get more data. Now, list all instances from the Task 5.4 together **with their father and mother**

In [ ]:
# _FILL_IN_YOUR_CODE_HERE


**Task 7** If you want to perform SPARQL queries, it is also possible. You can simply execute the query using `owlready2.default_world.sparql()` function.

If you have time, rewrite the query from Task 6 into SPARQL and execute against the ontology.

Use https://owlready2.readthedocs.io/en/latest/sparql.html#performing-sparql-queries for reference.

In [ ]:
# _FILL_IN_YOUR_CODE_HERE


There is much more to do with owlready2: inference, SWRL rules, ...  
Take a look at the [documentation](https://owlready2.readthedocs.io/en/latest/index.html) and the great **[Cooking with Python and KBpedia series](https://www.mkbergman.com/cooking-with-python-and-kbpedia/)**